In [17]:
from hypex.dataset import Dataset, InfoRole, TreatmentRole, TargetRole
from hypex.experiments.ab import ABTest

In [18]:
data = Dataset(
    roles={
        "user_id": InfoRole(int),
        "treat": TreatmentRole(),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole(), 
        "gender": TargetRole()
    }, data="data1.csv",
)
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0    0.0       488.0   414.444444   NaN      M   
1           1             8    1.0       512.5   462.222222  26.0    NaN   
2           2             7    1.0       483.0   479.444444  25.0      M   
3           3             0    1.0       501.5   424.333333  39.0      M   
4           4             1    2.0       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10    1.0       538.5   450.444444  42.0      M   
9996     9996             0    0.0       500.5   430.888889  26.0      F   
9997     9997             3    1.0       473.0   534.111111  22.0      F   
9998     9998             2    1.0       495.0   523.222222  67.0      F   
9999     9999             7    1.0       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [19]:
data.roles

{'user_id': Info(<class 'int'>),
 'treat': Treatment(<class 'float'>),
 'pre_spends': Target(<class 'float'>),
 'post_spends': Target(<class 'float'>),
 'gender': Target(<class 'str'>),
 'signup_month': Feature(<class 'int'>),
 'age': Feature(<class 'float'>),
 'industry': Feature(<class 'str'>)}

In [25]:
test = ABTest(additional_tests=['t-test', 'u-test', 'chi2-test'], multitest_method="quantile")
result = test.execute(data)

In [26]:
result.resume

       feature group TTest pass  TTest p-value UTest pass  UTest p-value  \
0   pre_spends     0     NOT OK   7.053045e-03         OK       0.070003   
1  post_spends     0     NOT OK   6.358305e-13         OK       0.109575   
2       gender     0        NaN            NaN        NaN            NaN   

  Chi2Test pass  Chi2Test p-value  
0           NaN               NaN  
1           NaN               NaN  
2            OK          0.828765  

In [27]:
result.multitest

         field accepted hypothesis
0   pre_spends                   3
0  post_spends                   2

In [28]:
result.difference

   control mean   test mean  difference  difference % group        field
0    484.898499  489.220115    4.321615      0.891241     1   pre_spends
0    484.898499  501.000000   16.101501      3.320592     1  post_spends
0    420.046779  483.463479   63.416700     15.097533     2   pre_spends
0    420.046779  440.571429   20.524650      4.886277     2  post_spends

In [24]:
result.sizes

   control size  test size  control size %  test size % group
0          4931       5061              49           50     1
0          4931          7              99            0     2